In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import re
from textblob import TextBlob
from wordcloud import WordCloud
import seaborn as sns
import matplotlib.pyplot as plt
import cufflinks as cf
%matplotlib inline 
cf.go_offline();

from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')
warnings.warn('this will not show ')

pd.set_option('display.max_columns',None)

In [2]:

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)


import plotly.graph_objects as go
 




In [3]:
df=pd.read_csv('amazon.csv')

In [4]:
df.head()

,Unnamed: 0,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
0,0,NaN,4,No issues.,23-07-2014,138,0,0,0,0,0.0,0.0
1,1,0mie,5,"Purchased this for my device, it worked as adv...",25-10-2013,409,0,0,0,0,0.0,0.0
2,2,1K3,4,it works as expected. I should have sprung for...,23-12-2012,715,0,0,0,0,0.0,0.0
3,3,1m2,5,This think has worked out great.Had a diff. br...,21-11-2013,382,0,0,0,0,0.0,0.0
4,4,2&amp;1/2Men,5,"Bought it with Retail Packaging, arrived legit...",13-07-2013,513,0,0,0,0,0.0,0.0


In [5]:
df.columns

Index(['Unnamed: 0', 'reviewerName', 'overall', 'reviewText', 'reviewTime',
       'day_diff', 'helpful_yes', 'helpful_no', 'total_vote',
       'score_pos_neg_diff', 'score_average_rating', 'wilson_lower_bound'],
      dtype='object')

In [6]:
df.shape

(4915, 12)

In [7]:
df=df.sort_values('wilson_lower_bound',ascending=False)
df.drop("Unnamed: 0",inplace=True, axis=1)
df.head()


,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,I have tested dozens of SDHC and micro-SDHC ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,NOTE: please read the last update (scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,"If your card gets hot enough to be painful, it...",09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,Sandisk announcement of the first 128GB micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [8]:
def missing_values_analysis(df):
    na_columns_=[col for col in df.columns if df[col].isnull().sum()>0]
    na_miss=df[na_columns_].isnull().sum().sort_values(ascending=True)
    ratio_=(df[na_columns_].isnull().sum()/ df.shape[0]*100).sort_values(ascending=True)
    missing_df=pd.concat([na_miss,np.round(ratio_,2)],axis=1,keys=['Missing Values','Ratio'])
    missing_df=pd.DataFrame(missing_df)
    return missing_df

In [9]:
def check_dataframe(df,head=5,tail=5):
    print('SHAPE'.center(82,'~'))
    print('Rows:{}'.format(df.shape[0]))
    print('Columns: {}'.format(df.shape[1]))
    print('TYPES'.center(82,'~'))
    print(df.dtypes)
    print("".center(82,"~"))
    print(missing_values_analysis(df))
    print('DUPLICATED VALUES'.center(82,"~"))
    print(df.duplicated().sum())
    print('QUANTILES'.center(82,"~"))
    print(df.quantile)

check_dataframe(df)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~SHAPE~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Rows:4915
Columns: 11
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~TYPES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
reviewerName             object
overall                   int64
reviewText               object
reviewTime               object
day_diff                  int64
helpful_yes               int64
helpful_no                int64
total_vote                int64
score_pos_neg_diff        int64
score_average_rating    float64
wilson_lower_bound      float64
dtype: object
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
              Missing Values  Ratio
reviewerName               1   0.02
reviewText                 1   0.02
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~DUPLICATED VALUES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
0
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~QUANTILES~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<bound method DataFrame.quantile of                  reviewerName  overall  \
2031     

In [10]:
def check_class(dataframe):
    nunique_df=pd.DataFrame({'variables':dataframe.columns,
                            'Classes':[dataframe[i].nunique()\
                                       for i in dataframe.columns]})
    nunique_df=nunique_df.sort_values('Classes',ascending = False)
    nunique_df=nunique_df.reset_index(drop=True)
    return nunique_df

check_class(df)

,variables,Classes
0,reviewText,4912
1,reviewerName,4594
2,reviewTime,690
3,day_diff,690
4,wilson_lower_bound,40
5,score_average_rating,28
6,score_pos_neg_diff,27
7,total_vote,26
8,helpful_yes,23
9,helpful_no,17


In [11]:
constraints=['#B34D22','#EBE00C','#1FEB0C','#0C92EB','#EB0CD5']

def categorical_variable_summary(df, column_name):
    fig = make_subplots(rows=1, cols=2,
                                        subplot_titles=('Countplot', 'Percentage'),
                                        specs=[[{"type": "bar"}, {'type': 'pie'}]])
    
    fig.add_trace(go.Bar(y=df[column_name].value_counts().values.tolist(),
                         x=[str(i) for i in df[column_name].value_counts().index],
                         text=df[column_name].value_counts().values.tolist(),
                         textfont=dict(size=14),
                         name=column_name,
                         textposition='auto',
                         showlegend=False,
                         marker=dict(color=constraints, line=dict(color='#DBE6EC', width=1))), row=1, col=1)
    
    fig.add_trace(go.Pie(labels=df[column_name].value_counts().keys(),
                         values=df[column_name].value_counts().values,
                         textfont=dict(size=18),
                         textposition='auto',
                         showlegend=False,
                         name=column_name,
                         marker=dict(colors=constraints)), row=1, col=2)
    
    fig.update_layout(title={'text': column_name,
                             'y': 0.9,
                             'x': 0.5,
                             'xanchor': 'center',
                             'yanchor': 'top'}, template='plotly_white')
    iplot(fig)



In [12]:
categorical_variable_summary(df,'overall')

In [13]:
df.reviewText.head()


2031    [[ UPDATE - 6/19/2014 ]]So my lovely wife boug...
3449    I have tested dozens of SDHC and micro-SDHC ca...
4212    NOTE:  please read the last update (scroll to ...
317     If your card gets hot enough to be painful, it...
4672    Sandisk announcement of the first 128GB micro ...
Name: reviewText, dtype: object

In [14]:
review_example=df.reviewText[317]
review_example

"If your card gets hot enough to be painful, it is defective and you need to contact SanDisk!From my experience, the larger micro SDs run hot on format and writing, but not to the pain threshold!I purchased one of these cards to use in a camera and Android tablet.  Great read speed but write speed was okay.  Used card for about two weeks and then all my files disappeared!  Tried to format the card and three different computers with XP, Windows, and Linux couldn't format it.  The camera and tablet couldn't format it.  I RMA'd it and got another because SanDisk is a good brand.  I did notice that when formatting or writing the card it got hot, very hot.Second card didn't last 24 hours and had the exact same symptoms as the first card.  Write a file or format it resulted in the card getting hot enough to reach the pain threshold.  I called up SanDisk and while explaining this was the second card I'd received with the exact same symptoms, when I mentioned it getting hot, they immediately i

In [15]:
review_example=re.sub("[^a-zA-Z]",' ',review_example)
review_example

'If your card gets hot enough to be painful  it is defective and you need to contact SanDisk From my experience  the larger micro SDs run hot on format and writing  but not to the pain threshold I purchased one of these cards to use in a camera and Android tablet   Great read speed but write speed was okay   Used card for about two weeks and then all my files disappeared   Tried to format the card and three different computers with XP  Windows  and Linux couldn t format it   The camera and tablet couldn t format it   I RMA d it and got another because SanDisk is a good brand   I did notice that when formatting or writing the card it got hot  very hot Second card didn t last    hours and had the exact same symptoms as the first card   Write a file or format it resulted in the card getting hot enough to reach the pain threshold   I called up SanDisk and while explaining this was the second card I d received with the exact same symptoms  when I mentioned it getting hot  they immediately i

In [16]:
review_example=review_example.lower().split()
review_example

['if',
 'your',
 'card',
 'gets',
 'hot',
 'enough',
 'to',
 'be',
 'painful',
 'it',
 'is',
 'defective',
 'and',
 'you',
 'need',
 'to',
 'contact',
 'sandisk',
 'from',
 'my',
 'experience',
 'the',
 'larger',
 'micro',
 'sds',
 'run',
 'hot',
 'on',
 'format',
 'and',
 'writing',
 'but',
 'not',
 'to',
 'the',
 'pain',
 'threshold',
 'i',
 'purchased',
 'one',
 'of',
 'these',
 'cards',
 'to',
 'use',
 'in',
 'a',
 'camera',
 'and',
 'android',
 'tablet',
 'great',
 'read',
 'speed',
 'but',
 'write',
 'speed',
 'was',
 'okay',
 'used',
 'card',
 'for',
 'about',
 'two',
 'weeks',
 'and',
 'then',
 'all',
 'my',
 'files',
 'disappeared',
 'tried',
 'to',
 'format',
 'the',
 'card',
 'and',
 'three',
 'different',
 'computers',
 'with',
 'xp',
 'windows',
 'and',
 'linux',
 'couldn',
 't',
 'format',
 'it',
 'the',
 'camera',
 'and',
 'tablet',
 'couldn',
 't',
 'format',
 'it',
 'i',
 'rma',
 'd',
 'it',
 'and',
 'got',
 'another',
 'because',
 'sandisk',
 'is',
 'a',
 'good',
 'br

In [17]:
rt=lambda x:re.sub("[^a-zA-Z]",' ',str(x))
df['reviewText']=df['reviewText'].map(rt)
df['reviewText']=df['reviewText'].str.lower()
df.head()

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109


In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
df[['polarity', 'subjectivity']] = df['reviewText'].apply(lambda Text:pd.Series(TextBlob(Text).sentiment))


for index, row in df['reviewText'].items():
    score=SentimentIntensityAnalyzer().polarity_scores(row)

    neg=score['neg']
    neu=score['neu']
    pos=score['pos']

    if neg>pos:
        df.loc[index,'sentiment']='Negative'
    elif pos>neg:
        df.loc[index,'sentiment']='Positive'
    else:
        df.loc[index,'sentiment']='Neutral'

In [19]:

df[df['sentiment']=='Positive'].sort_values('wilson_lower_bound',ascending=False).head(5)

,reviewerName,overall,reviewText,reviewTime,day_diff,helpful_yes,helpful_no,total_vote,score_pos_neg_diff,score_average_rating,wilson_lower_bound,polarity,subjectivity,sentiment
2031,"Hyoun Kim ""Faluzure""",5,update so my lovely wife boug...,05-01-2013,702,1952,68,2020,1884,0.966337,0.957544,0.163859,0.562259,Positive
3449,NLee the Engineer,5,i have tested dozens of sdhc and micro sdhc ca...,26-09-2012,803,1428,77,1505,1351,0.948837,0.936519,0.103870,0.516435,Positive
4212,SkincareCEO,1,note please read the last update scroll to ...,08-05-2013,579,1568,126,1694,1442,0.925620,0.912139,0.212251,0.505394,Positive
317,"Amazon Customer ""Kelly""",1,if your card gets hot enough to be painful it...,09-02-2012,1033,422,73,495,349,0.852525,0.818577,0.143519,0.494207,Positive
4672,Twister,5,sandisk announcement of the first gb micro ...,03-07-2014,158,45,4,49,41,0.918367,0.808109,0.172332,0.511282,Positive


In [20]:
categorical_variable_summary(df,'sentiment')